In [1]:
pip install cfbd --upgrade

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at /private/var/containers/Bundle/Application/1156DF36-3970-466E-AEBB-AD8D32B96E4C/Carnets-sci.app/Library/lib/python3.11/site-packages/statsmodels-0.13.5-py3.11-macosx-11.5-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /private/var/containers/Bundle/Application/1156DF36-3970-466E-AEBB-AD8D32B96E4C/Carnets-sci.app/Library/lib/python3.11/site-packages/jupyterlab-3.4.8-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /private/var/containers/Bundle/Application/1156DF36-3970-466E-AEBB-AD8D32B96E4C/Carnets-sci.app/Library/lib/python3.11/site-packages/rasterio-1.3.4-py3.11-macosx-11.5-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading eg


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cfbd
import pandas as pd
import numpy as np

In [3]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = "insert API key, not mine :)"
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

wk = 14

In [4]:
response = games = cfbd.GamesApi(api_config).get_games(year=2023, season_type='both', division = 'fbs')
data = pd.DataFrame.from_records([g.to_dict() for g in games])

for index,row in data.iterrows():
    if(data.loc[index,'home_division']!='fbs'):
        data.loc[index,'home_team'] = 'FCS'
    elif(data.loc[index,'away_division']!='fbs'):
        data.loc[index,'away_team'] = 'FCS'

data = data[
    (data['home_points'] == data['home_points'])
    & (data['away_points'] == data['away_points'])
    & (pd.notna(data['home_conference']))
    & (pd.notna(data['away_conference']))
]

In [5]:
data['home_spread'] = np.where(data['neutral_site'] == True, data['home_points'] - data['away_points'], (data['home_points'] - data['away_points'] - 0))
data['away_spread'] = -data['home_spread']

In [6]:
teams = pd.concat([
    data[['home_team', 'home_spread', 'away_team']].rename(columns={'home_team': 'team', 'home_spread': 'spread', 'away_team': 'opponent'}),
    data[['away_team', 'away_spread', 'home_team']].rename(columns={'away_team': 'team', 'away_spread': 'spread', 'home_team': 'opponent'})
])

In [7]:
teams['spread'] = np.where(teams['spread'] > 1, 1, teams['spread'])
teams['spread'] = np.where(teams['spread'] < -1, -1, teams['spread'])
#teams['spread'] = np.where(teams['spread'] > -3 and teams['spread'] < 0, -3, teams['spread'])
#teams['spread'] = np.where(teams['spread'] < 3 and teams['spread'] > 0, 3, teams['spread'])

In [8]:
spreads = teams.groupby('team').spread.mean()

In [9]:
terms = []
solutions = []
tm_opps = ['Ohio State']
tm2_opps = ['Tennessee']
for team in spreads.keys():
    row = []
    opps = list(teams[teams['team'] == team]['opponent'])
    if team == tm_opps[0]:
        tm_opps += opps
    if team == tm2_opps[0]:
        tm2_opps += opps
    for opp in spreads.keys():
        if opp == team:
            row.append(1)
        elif opp in opps:
            row.append(-1.0/len(opps))
        else:
            row.append(0)
    terms.append(row)
    solutions.append(spreads[team])

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

solution = np.linalg.solve(np.array(terms), np.array(solutions))
ratings = list(zip(spreads.keys(), solution))
srs = pd.DataFrame(ratings, columns=['team', 'rating'])
srs = srs.set_index(srs['team'])
srs = pd.concat([srs,spreads], axis = 1)

srs['sos'] = srs['rating'] - srs['spread']
srs[["sos"]] = scaler.fit_transform(srs[["sos"]])
mean = srs.rating.mean()
srs['rating'] = srs['rating'] - mean
srs[["rating"]] = scaler.fit_transform(srs[["rating"]])
srs['spread'] = (srs['spread']+1)/2
#display(srs.sort_values('rating', ascending=False).iloc[:])
display(srs.sort_values('rating', ascending=False).loc[tm_opps])
display(srs.sort_values('rating', ascending=False).loc[tm2_opps])

,team,rating,spread,sos
team,,,,
Ohio State,Ohio State,0.972167,1.000000,0.666919
FCS,FCS,0.077344,0.036364,0.568456
Western Kentucky,Western Kentucky,0.531329,0.571429,0.522635
Maryland,Maryland,0.625518,0.714286,0.447063
Penn State,Penn State,0.812416,0.857143,0.588808
Indiana,Indiana,0.392308,0.285714,0.789478
Notre Dame,Notre Dame,0.786635,0.750000,0.750645
Purdue,Purdue,0.482118,0.285714,1.000000


,team,rating,spread,sos
team,,,,
Tennessee,Tennessee,0.689027,0.714286,0.595932
Virginia,Virginia,0.412790,0.285714,0.837488
FCS,FCS,0.077344,0.036364,0.568456
UT San Antonio,UT San Antonio,0.448927,0.571429,0.329477
South Carolina,South Carolina,0.431221,0.285714,0.880692
Texas A&M,Texas A&M,0.567037,0.571429,0.606336
Florida,Florida,0.648745,0.714286,0.501508
Alabama,Alabama,0.909623,0.875000,0.779626


In [11]:
### data = pd.DataFrame.from_records([g.to_dict() for g in games])

for index,row in data.iterrows():
    if(data.loc[index,'home_division']!='fbs'):
        data.loc[index,'home_team'] = 'FCS'
    elif(data.loc[index,'away_division']!='fbs'):
        data.loc[index,'away_team'] = 'FCS'

data = data[
    (data['home_points'] == data['home_points'])
    & (data['away_points'] == data['away_points'])
    & (pd.notna(data['home_conference']))
    & (pd.notna(data['away_conference']))
]

data['home_spread'] = np.where(data['neutral_site'] == True, data['home_points'] - data['away_points'], (data['home_points'] - data['away_points'] - 3))
data['away_spread'] = -data['home_spread']

teams = pd.concat([
    data[['home_team', 'home_spread', 'away_team']].rename(columns={'home_team': 'team', 'home_spread': 'spread', 'away_team': 'opponent'}),
    data[['away_team', 'away_spread', 'home_team']].rename(columns={'away_team': 'team', 'away_spread': 'spread', 'home_team': 'opponent'})
])

teams['spread'] = np.where(teams['spread'] > 25, 25, teams['spread'])
teams['spread'] = np.where(teams['spread'] < -25, -25, teams['spread'])
#teams['spread'] = np.where(teams['spread'] > -3 and teams['spread'] < 0, -3, teams['spread'])
#teams['spread'] = np.where(teams['spread'] < 3 and teams['spread'] > 0, 3, teams['spread'])

spreads = teams.groupby('team').spread.mean()

terms = []
solutions = []
tm_opps = ['LSU']
tm2_opps = ['Wake Forest']
for team in spreads.keys():
    row = []
    opps = list(teams[teams['team'] == team]['opponent'])
    if team == tm_opps[0]:
        tm_opps += opps
    if team == tm2_opps[0]:
        tm2_opps += opps
    for opp in spreads.keys():
        if opp == team:
            row.append(1)
        elif opp in opps:
            row.append(-1.0/len(opps))
        else:
            row.append(0)
    terms.append(row)
    solutions.append(spreads[team])

solution = np.linalg.solve(np.array(terms), np.array(solutions))
ratings = list(zip(spreads.keys(), solution))
srs2 = pd.DataFrame(ratings, columns=['team', 'SRS_rating'])
srs2 = srs2.set_index(srs2['team'])
srs2 = pd.concat([srs2,spreads], axis = 1)
srs2.rename(columns={'spread':'mov'},inplace=True)
srs2['SOS'] = srs2['SRS_rating'] - srs2['mov']
mean = srs2.SRS_rating.mean()
srs2['SRS_rating'] = srs2['SRS_rating'] - mean
srs2['SOS'] = srs2['SOS'] - mean
srs2[["SRS_rating"]] = scaler.fit_transform(srs2[["SRS_rating"]])
srs2[["SOS"]] = scaler.fit_transform(srs2[["SOS"]])
#display(srs2.sort_values('SRS_rating', ascending=False).iloc[:25])
display(srs2.sort_values('SRS_rating', ascending=False).loc[tm_opps])
display(srs2.sort_values('SRS_rating', ascending=False).loc[tm2_opps])

,team,SRS_rating,mov,SOS
team,,,,
LSU,LSU,0.804875,11.125000,0.862073
FCS,FCS,0.018962,-18.427273,0.636054
Arkansas,Arkansas,0.554571,1.375000,0.810150
Auburn,Auburn,0.512117,-0.285714,0.801761
Army,Army,0.350550,-4.000000,0.615138
Florida State,Florida State,0.948539,17.142857,0.866837
Mississippi State,Mississippi State,0.476337,-1.142857,0.762484
Ole Miss,Ole Miss,0.787229,10.428571,0.858949
Missouri,Missouri,0.693965,9.500000,0.679069


,team,SRS_rating,mov,SOS
team,,,,
Wake Forest,Wake Forest,0.373293,0.571429,0.401072
FCS,FCS,0.018962,-18.427273,0.636054
Vanderbilt,Vanderbilt,0.242858,-8.125000,0.588648
Georgia Tech,Georgia Tech,0.436326,-1.571429,0.687107
Pittsburgh,Pittsburgh,0.404797,-1.857143,0.624622
Old Dominion,Old Dominion,0.287898,-1.714286,0.321581
Clemson,Clemson,0.651595,5.571429,0.805516
Virginia Tech,Virginia Tech,0.476380,-0.714286,0.737152


In [12]:
w1 = 1.3
w2 = 0.6
w3 = 1.2
sm = 0.28

ranks = pd.concat([srs.drop(columns=['team']),srs2.drop(columns=['team'])],axis=1)
ranks['avg_SOS'] = (0.83*ranks['sos']+0.17*ranks['SOS'])
ranks['rank'] = ((ranks['rating']+0.5)*w1+(ranks['SRS_rating']+0.5)*w2+((ranks['spread']+0.5)*(sm*ranks['avg_SOS']+(1-sm/2)))*w3)/3
scal = MinMaxScaler(feature_range=(50,100))
mm = np.array([0.4766666,1.57]).reshape(-1,1)
scal.fit(mm)
ranks[['rank']] = scal.transform(ranks[['rank']])
ranks.sort_values('rank',ascending=False,inplace=True)
ranks.reset_index(inplace=True)
pd.set_option('display.max_rows',None)

grades = ['A++','A+',"A","A-",'B+','B','B-','C+',"C","C-",'D+',"D","D-","F+","F","F-",'F--']
grades.reverse()
bins = [0]
g = 50
while(g<101):
    bins.append(round(g,12))
    g+=(10/3)
bins.append(200)
ranks['grade'] = pd.cut(ranks['rank'],bins=bins,labels=grades,)
ranks.index = ranks.index + 1
ranks[['team','rating','SRS_rating','avg_SOS','rank']].loc[:]
ranks[['team','grade']].loc[:]
from tabulate import tabulate
#display in plain text format
#print(tabulate(ranks[['team','grade']], headers='keys', tablefmt='html'))

/private/var/containers/Bundle/Application/1156DF36-3970-466E-AEBB-AD8D32B96E4C/Carnets-sci.app/Library/lib/python3.11/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [13]:
ranks[['team','grade']].loc[:]

,team,grade
1,Florida State,A++
2,Ohio State,A+
3,Michigan,A+
4,Oklahoma,A+
5,Texas,A+
6,Washington,A
7,Alabama,A
8,James Madison,A
9,Ole Miss,A-
10,Utah,A-


In [14]:
rankings = srs.sort_values('rating', ascending=False).reset_index(drop=True)[['team','rating','sos']]
teams = cfbd.TeamsApi(api_config)
fbs = teams.get_fbs_teams()
fbs_n = list(team.school for team in fbs)
#rankings = rankings[rankings['team'].isin(fbs_n)]

In [15]:
stats_api = cfbd.StatsApi(api_config)
s = stats_api.get_team_season_stats(year=2022,end_week=wk)

In [16]:
df = pd.DataFrame.from_records([si.to_dict() for si in s])
df = df.drop(columns = ['season','conference'])
ndf = pd.DataFrame(index = df.team.unique(),columns = df.stat_name.unique())
for index,row in df.iterrows():
    ndf.loc[row[0],row[1]] = df.iloc[index,2]
del(df)
ndf = ndf.sort_index()

In [17]:
ypp = ndf[["netPassingYards",'passAttempts','rushingYards','rushingAttempts']]
ypp['YPA'] = ypp['netPassingYards']/ypp['passAttempts']
ypp['YPC'] = ypp['rushingYards']/ypp['rushingAttempts']
ypp['passPct'] = ypp['passAttempts']/(ypp['passAttempts']+ypp['rushingAttempts'])
ypp['YPP'] = ypp['YPA']*ypp['passPct'] + ypp['YPC']*(1-ypp['passPct'])
ypp = ypp.drop(columns = ["netPassingYards",'passAttempts','rushingYards','rushingAttempts'])
#ypp.head()

/private/var/mobile/Containers/Data/Application/3C496577-619B-439B-B964-E1950AF27C16/tmp/ipykernel_11917/767249118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ypp['YPA'] = ypp['netPassingYards']/ypp['passAttempts']
/private/var/mobile/Containers/Data/Application/3C496577-619B-439B-B964-E1950AF27C16/tmp/ipykernel_11917/767249118.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ypp['YPC'] = ypp['rushingYards']/ypp['rushingAttempts']
/private/var/mobile/Containers/Data/Application/3C496577-619B-439B

In [18]:
#defense

In [19]:
#concat

In [20]:
#formulas